Ce code permet de faire l'interférence

Après vous pouvez utiliser l'autre pour la visualisation

In [1]:
import sys

# Ajouter le chemin de ton projet local en tête
sys.path.insert(0, r"D:\ETUDEµ\CITE\ETAPE4\PROJET DE FIN DE SESSION\TOURMANT 1")

from db.pool import init_pools

from inference.video_inference_prod import run_video_production

init_pools()


run_video_production("data/test.mp4",site_id=1, camera_id=1,model_path="../models/best_yolo11x.pt")



Pour faire les visualisation de la video et objet détectecté

In [3]:
import cv2
from datetime import timedelta
import sys

# Ajouter le chemin de ton projet local en tête
sys.path.insert(0, r"D:\ETUDEµ\CITE\ETAPE4\PROJET DE FIN DE SESSION\TOURMANT 1")

from inference.persistence import get_detections_for_frame


def replay_video_with_boxes(
    video_path: str,
    camera_id: int,
    start_time,
    fps: int = 25,
    slow_factor: float = 1.0
):
    """
    Replay vidéo avec affichage des bounding boxes
    STOCKÉES EN BASE (aucune détection YOLO).

    slow_factor:
        1.0 = normal
        2.0 = 2x plus lent
        4.0 = 4x plus lent
    """

    cap = cv2.VideoCapture(video_path)
    frame_duration = timedelta(seconds=1 / fps)

    current_time = start_time
    frame_idx = 0

    wait_ms = int((1000 / fps) * slow_factor)

    print(f"[INFO] Replay fps={fps} | slow_factor={slow_factor}")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        next_time = current_time + frame_duration

        detections = get_detections_for_frame(
            camera_id=camera_id,
            start_ts=current_time,
            end_ts=next_time
        )

        for det in detections:
            x1 = int(det["bbox_x"])
            y1 = int(det["bbox_y"])
            x2 = int(x1 + det["bbox_w"])
            y2 = int(y1 + det["bbox_h"])

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

            cv2.putText(
                frame,
                f'{det["object_class"]} #{det["track_id"]}',
                (x1, y1 - 5),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 255, 0),
                1
            )

        cv2.imshow("Replay detections", frame)

        if cv2.waitKey(wait_ms) & 0xFF == ord("q"):
            break

        current_time = next_time
        frame_idx += 1

    cap.release()
    cv2.destroyAllWindows()


Test. Pour vérifier les objets détecter . Il faut renseigné la date et l'heure du debut et de fin

In [4]:
from datetime import datetime

video_start_time = datetime.strptime(
    "2026-01-18 20:31:40",
    "%Y-%m-%d %H:%M:%S"
)

from db.pool import init_pools

init_pools()
replay_video_with_boxes(
    "data/test.mp4",
    camera_id=1,
    start_time=video_start_time,
    fps=25,
    slow_factor=1.0
)




[INFO] Replay fps=25 | slow_factor=1.0


In [3]:
import torch

# Charger le modèle
model = torch.load("../models/best_yolo11x.pt")

# Vérifier si les classes sont dans l'objet
if hasattr(model, 'names'):
    print("Classes :", model.names)
elif hasattr(model, 'model') and hasattr(model.model, 'names'):
    print("Classes :", model.model.names)
else:
    print("Aucune info de classes trouvée dans le modèle.")


C:\Users\MEHOUN MARTINIEN\AppData\Local\Temp\ipykernel_24800\930000970.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load("../models/best_yolo11x.pt")


Aucune info de classes trouvée dans le modèle.


In [4]:
from ultralytics import YOLO

model = YOLO("../models/best_yolo11x.pt")
print(model.names)

{0: 'Hardhat', 1: 'Mask', 2: 'NO-Hardhat', 3: 'NO-Mask', 4: 'NO-Safety Vest', 5: 'Person', 6: 'Safety Cone', 7: 'Safety Vest', 8: 'machinery', 9: 'utility pole', 10: 'vehicle'}
